## DIA 026: Implementacion de la Funcionalidad de Restablecimiento de Contraseña


1. Objetivos del Día 26
Implementar la Funcionalidad de Restablecimiento de Contraseña:

Permitir a los usuarios solicitar un enlace para restablecer su contraseña.
Facilitar el proceso de establecimiento de una nueva contraseña mediante un enlace seguro enviado por email.
Mejorar la Seguridad:

Asegurar que los tokens de restablecimiento sean seguros y expiren después de un tiempo determinado.
Validar adecuadamente las solicitudes para prevenir abusos.
Actualizar el Frontend:

Añadir formularios y páginas necesarios para solicitar y realizar el restablecimiento de contraseña.
Actualizar las Pruebas Automatizadas:

Garantizar que la funcionalidad de restablecimiento de contraseña funciona correctamente y de manera segura.
2. Configurar Funcionalidades de Restablecimiento de Contraseña
2.1. Actualizar el Modelo de Usuario
Aunque el modelo User ya cuenta con campos esenciales, no es necesario agregar nuevos campos para la funcionalidad de restablecimiento de contraseña, ya que utilizaremos tokens temporales generados dinámicamente.

Sin embargo, si deseas llevar un seguimiento de solicitudes de restablecimiento o limitar el número de intentos, podrías considerar añadir campos adicionales. Para este ejemplo, procederemos sin cambios en el modelo.

2.2. Crear Funciones para Generar y Verificar Tokens de Restablecimiento
Añade funciones auxiliares para manejar la generación y verificación de tokens de restablecimiento de contraseña utilizando itsdangerous.

python
Copiar
# api.py

from itsdangerous import URLSafeTimedSerializer, SignatureExpired, BadSignature

# Serializer ya configurado para confirmación de email
# Puedes reutilizar el mismo serializer o crear uno nuevo si lo prefieres
def generate_reset_token(email):
    return s.dumps(email, salt='password-reset')

def confirm_reset_token(token, expiration=3600):
    try:
        email = s.loads(token, salt='password-reset', max_age=expiration)
    except SignatureExpired:
        return None  # Token expirado
    except BadSignature:
        return None  # Token inválido
    return email
Explicación:

generate_reset_token(email): Genera un token seguro basado en el email del usuario con un salt específico para restablecimiento de contraseña.
confirm_reset_token(token, expiration): Verifica la validez y expiración del token. Retorna el email si es válido; de lo contrario, retorna None.
3. Implementar Endpoints de Restablecimiento de Contraseña
3.1. Endpoint para Solicitar Restablecimiento de Contraseña
Crea un endpoint /reset_password_request que permita a los usuarios solicitar un enlace para restablecer su contraseña.

python
Copiar
# api.py

@app.route('/reset_password_request', methods=['POST'])
def reset_password_request():
    """
    Endpoint para solicitar el restablecimiento de contraseña.
    Envía un email con un enlace para restablecer la contraseña.
    """
    data = request.get_json()
    email = data.get('email', None)
    
    if not email:
        return jsonify({"msg": "Email requerido"}), 400
    
    user = User.query.filter_by(email=email).first()
    if not user:
        return jsonify({"msg": "Si el email está registrado, recibirás un enlace para restablecer tu contraseña."}), 200  # Evita exponer si el email existe
    
    # Generar el token de restablecimiento
    token = generate_reset_token(user.email)
    
    # Crear el enlace de restablecimiento
    reset_url = url_for('reset_password', token=token, _external=True)
    html = render_template('reset_password_email.html', reset_url=reset_url)
    subject = "Restablece tu contraseña"
    
    # Enviar el email de restablecimiento
    msg = Message(recipients=[user.email],
                  subject=subject,
                  html=html)
    mail.send(msg)
    
    return jsonify({"msg": "Si el email está registrado, recibirás un enlace para restablecer tu contraseña."}), 200
Explicación:

Validación del Email: Verifica que se haya proporcionado un email en la solicitud.
Búsqueda del Usuario: Busca al usuario asociado al email. Si no existe, devuelve un mensaje genérico para evitar exponer la existencia del email.
Generación del Token y Envío del Email: Si el usuario existe, genera un token de restablecimiento y envía un email con un enlace que contiene el token.
3.2. Endpoint para Restablecer la Contraseña
Crea un endpoint /reset_password/<token> que permita a los usuarios establecer una nueva contraseña utilizando el token recibido por email.

python
Copiar
# api.py

from datetime import datetime

@app.route('/reset_password/<token>', methods=['POST'])
def reset_password(token):
    """
    Endpoint para restablecer la contraseña utilizando un token válido.
    """
    data = request.get_json()
    new_password = data.get('password', None)
    
    if not new_password:
        return jsonify({"msg": "Nueva contraseña requerida"}), 400
    
    # Confirmar el token
    email = confirm_reset_token(token)
    if not email:
        return jsonify({"msg": "El enlace de restablecimiento es inválido o ha expirado."}), 400
    
    user = User.query.filter_by(email=email).first_or_404()
    
    # Actualizar la contraseña
    hashed_password = bcrypt.generate_password_hash(new_password).decode('utf-8')
    user.password = hashed_password
    db.session.commit()
    
    return jsonify({"msg": "Tu contraseña ha sido actualizada exitosamente."}), 200
Explicación:

Validación de la Nueva Contraseña: Asegura que se haya proporcionado una nueva contraseña.
Verificación del Token: Utiliza confirm_reset_token para verificar la validez del token. Si es inválido o ha expirado, retorna un error.
Actualización de la Contraseña: Si el token es válido, actualiza la contraseña del usuario con el nuevo hash generado.
4. Actualizar el Frontend para Manejar el Restablecimiento de Contraseña
4.1. Crear Plantillas HTML para Restablecimiento de Contraseña
Crea una nueva plantilla reset_password_email.html en el directorio templates para el contenido del email de restablecimiento de contraseña.

html
Copiar
<!-- templates/reset_password_email.html -->

<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Restablece tu Contraseña</title>
</head>
<body>
    <p>Hola,</p>
    <p>Has solicitado restablecer tu contraseña. Por favor, haz clic en el siguiente enlace para establecer una nueva contraseña:</p>
    <p><a href="{{ reset_url }}">{{ reset_url }}</a></p>
    <p>Si no solicitaste este cambio, por favor ignora este correo.</p>
</body>
</html>
Explicación:

Contenido del Email:
Instrucciones para restablecer la contraseña.
Enlace seguro que contiene el token de restablecimiento.
Mensaje de precaución en caso de recibir el email por error.
4.2. Actualizar index.html para Incluir Formularios de Restablecimiento
Actualiza el archivo templates/index.html para incluir formularios que permitan a los usuarios solicitar y realizar el restablecimiento de contraseña.

html
Copiar
<!-- templates/index.html -->

<!DOCTYPE html>
<html>
<head>
    <title>Clasificador de Dígitos MNIST</title>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>

    <h1>Clasificador de Dígitos MNIST</h1>
    
    <!-- Formulario de Registro -->
    <h2>Registrar Usuario</h2>
    <form id="registerForm">
        <label for="reg_username">Usuario:</label>
        <input type="text" id="reg_username" name="username" required>
        <br>
        <label for="reg_email">Email:</label>
        <input type="email" id="reg_email" name="email" required>
        <br>
        <label for="reg_password">Contraseña:</label>
        <input type="password" id="reg_password" name="password" required>
        <br>
        <button type="submit">Registrar</button>
    </form>
    
    <div id="registerResult"></div>
    
    <!-- Formulario de Login -->
    <h2>Iniciar Sesión</h2>
    <form id="loginForm">
        <label for="username">Usuario:</label>
        <input type="text" id="username" name="username" required>
        <br>
        <label for="password">Contraseña:</label>
        <input type="password" id="password" name="password" required>
        <br>
        <button type="submit">Iniciar Sesión</button>
    </form>
    
    <div id="loginResult"></div>
    
    <!-- Formulario de Solicitud de Restablecimiento de Contraseña -->
    <h2>¿Olvidaste tu Contraseña?</h2>
    <form id="resetRequestForm">
        <label for="reset_email">Ingresa tu Email:</label>
        <input type="email" id="reset_email" name="email" required>
        <br>
        <button type="submit">Restablecer Contraseña</button>
    </form>
    
    <div id="resetRequestResult"></div>
    
    <!-- Formulario de Restablecimiento de Contraseña -->
    <h2>Restablecer Contraseña</h2>
    <form id="resetPasswordForm">
        <label for="new_password">Nueva Contraseña:</label>
        <input type="password" id="new_password" name="password" required>
        <br>
        <button type="submit">Actualizar Contraseña</button>
    </form>
    
    <div id="resetPasswordResult"></div>
    
    <!-- Formulario de Predicción -->
    <h2>Enviar Imagen para Predicción</h2>
    <input type="file" id="fileInput" accept="image/*">
    <button onclick="uploadImage()">Enviar para Predicción</button>

    <h2>Resultado:</h2>
    <p id="result"></p>

    <script>
        let accessToken = '';

        // Manejar el formulario de registro
        $('#registerForm').submit(function(event) {
            event.preventDefault();
            const username = $('#reg_username').val();
            const email = $('#reg_email').val();
            const password = $('#reg_password').val();

            $.ajax({
                url: '/register',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, email: email, password: password }),
                success: function(response) {
                    $('#registerResult').text('Registro exitoso. Por favor, confirma tu email.');
                },
                error: function(xhr, status, error) {
                    $('#registerResult').text('Error en el registro: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Manejar el formulario de login
        $('#loginForm').submit(function(event) {
            event.preventDefault();
            const username = $('#username').val();
            const password = $('#password').val();

            $.ajax({
                url: '/login',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, password: password }),
                success: function(response) {
                    accessToken = response.access_token;
                    $('#loginResult').text('Inicio de sesión exitoso. Token obtenido.');
                },
                error: function(xhr, status, error) {
                    $('#loginResult').text('Error en el inicio de sesión: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Manejar el formulario de solicitud de restablecimiento de contraseña
        $('#resetRequestForm').submit(function(event) {
            event.preventDefault();
            const email = $('#reset_email').val();

            $.ajax({
                url: '/reset_password_request',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ email: email }),
                success: function(response) {
                    $('#resetRequestResult').text(response.msg);
                },
                error: function(xhr, status, error) {
                    $('#resetRequestResult').text('Error: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Manejar el formulario de restablecimiento de contraseña
        $('#resetPasswordForm').submit(function(event) {
            event.preventDefault();
            const urlParams = new URLSearchParams(window.location.search);
            const token = window.location.pathname.split('/').pop();
            const new_password = $('#new_password').val();

            $.ajax({
                url: `/reset_password/${token}`,
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ password: new_password }),
                success: function(response) {
                    $('#resetPasswordResult').text(response.msg);
                },
                error: function(xhr, status, error) {
                    $('#resetPasswordResult').text('Error: ' + xhr.responseJSON.msg);
                }
            });
        });

        function uploadImage() {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            var fileInput = document.getElementById('fileInput');
            var file = fileInput.files[0];
            if (!file) {
                alert("Por favor, selecciona una imagen.");
                return;
            }

            var formData = new FormData();
            formData.append('file', file);

            $.ajax({
                url: '/predict',
                type: 'POST',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                data: formData,
                contentType: false,
                processData: false,
                success: function(response) {
                    if(response.error){
                        $('#result').text('Error: ' + response.error);
                    } else {
                        let cacheStatus = response.cached ? ' (Desde Cache)' : '';
                        $('#result').text('Predicción: ' + response.prediccion + ' | Probabilidad: ' + (response.probabilidad * 100).toFixed(2) + '%' + cacheStatus);
                    }
                },
                error: function(xhr, status, error) {
                    $('#result').text('Error: ' + xhr.responseJSON.error);
                }
            });
        }
    </script>

</body>
</html>
Explicación:

Formulario de Solicitud de Restablecimiento de Contraseña:
Permite a los usuarios ingresar su email para solicitar un enlace de restablecimiento.
Formulario de Restablecimiento de Contraseña:
Permite a los usuarios establecer una nueva contraseña utilizando el token de restablecimiento.
Interacción con los Endpoints:
Maneja las solicitudes de restablecimiento de contraseña y la actualización de la contraseña mediante AJAX.
Nota Importante:
Para manejar la ruta del token en el frontend, es necesario que la página de restablecimiento de contraseña (reset_password/<token>) cargue la plantilla correspondiente. En este ejemplo simplificado, el formulario de restablecimiento está en la misma página, pero en una implementación real, podrías crear una página dedicada para ello.
5. Actualizar las Pruebas Automatizadas para la Funcionalidad de Restablecimiento
Es esencial asegurarse de que la funcionalidad de restablecimiento de contraseña funcione correctamente mediante pruebas automatizadas.

5.1. Actualizar test_api.py para Incluir Pruebas de Restablecimiento de Contraseña
Añade las siguientes pruebas en tests/test_api.py:

python
Copiar
# tests/test_api.py

import pytest
from api import app, db, User, generate_reset_token, confirm_reset_token
import os
from io import BytesIO
from PIL import Image
import numpy as np
import json
from flask import url_for
from unittest.mock import patch

@pytest.fixture
def client():
    app.config['TESTING'] = True
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'  # Usar una base de datos en memoria para pruebas
    app.config['MAIL_SUPPRESS_SEND'] = True  # Evita el envío de emails durante las pruebas
    with app.test_client() as client:
        with app.app_context():
            db.create_all()
            yield client
            db.session.remove()
            db.drop_all()

def generate_image(digit=5):
    # Genera una imagen en blanco con un dígito negro en el centro
    img = Image.new('L', (28, 28), color=255)
    # Aquí podrías dibujar el dígito, pero para simplicidad, dejaremos la imagen en blanco
    # En un caso real, usarías una imagen real de MNIST
    return img

def register_user(client, username, email, password):
    response = client.post('/register', json={
        'username': username,
        'email': email,
        'password': password
    })
    return response

def login(client, username, password):
    response = client.post('/login', json={
        'username': username,
        'password': password
    })
    data = response.get_json()
    return data.get('access_token')

@patch('api.mail.send')
def test_reset_password_request(mock_send, client):
    # Registrar un nuevo usuario
    register_user(client, 'testuser', 'test@example.com', 'testpassword')
    
    # Solicitar restablecimiento de contraseña
    response = client.post('/reset_password_request', json={'email': 'test@example.com'})
    assert response.status_code == 200
    json_data = response.get_json()
    assert json_data['msg'] == 'Si el email está registrado, recibirás un enlace para restablecer tu contraseña.'
    
    # Verificar que se envió un email de restablecimiento
    assert mock_send.called
    msg = mock_send.call_args[0][0]
    assert msg.recipients == ['test@example.com']
    assert 'Restablece tu contraseña' in msg.subject
    assert 'reset_url' in msg.html

def test_reset_password_success(client):
    # Registrar un nuevo usuario
    register_user(client, 'testuser', 'test@example.com', 'testpassword')
    
    # Generar un token de restablecimiento
    token = generate_reset_token('test@example.com')
    
    # Restablecer la contraseña
    response = client.post(f'/reset_password/{token}', json={'password': 'newpassword'})
    assert response.status_code == 200
    json_data = response.get_json()
    assert json_data['msg'] == 'Tu contraseña ha sido actualizada exitosamente.'
    
    # Iniciar sesión con la nueva contraseña
    token_login = login(client, 'testuser', 'newpassword')
    assert token_login is not None

def test_reset_password_invalid_token(client):
    # Intentar restablecer la contraseña con un token inválido
    response = client.post('/reset_password/invalidtoken', json={'password': 'newpassword'})
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['msg'] == 'El enlace de restablecimiento es inválido o ha expirado.'

def test_reset_password_expired_token(client):
    # Generar un token que expirará inmediatamente
    with patch('api.confirm_reset_token', return_value=None):
        response = client.post('/reset_password/sometoken', json={'password': 'newpassword'})
        assert response.status_code == 400
        json_data = response.get_json()
        assert json_data['msg'] == 'El enlace de restablecimiento es inválido o ha expirado.'

def test_reset_password_user_not_found(client):
    # Generar un token para un email no registrado
    token = generate_reset_token('nonexistent@example.com')
    
    # Intentar restablecer la contraseña
    response = client.post(f'/reset_password/{token}', json={'password': 'newpassword'})
    assert response.status_code == 404  # Usuario no encontrado
Explicación:

test_reset_password_request: Verifica que al solicitar el restablecimiento de contraseña, se envíe un email correctamente.
test_reset_password_success: Asegura que un usuario pueda restablecer su contraseña utilizando un token válido y luego iniciar sesión con la nueva contraseña.
test_reset_password_invalid_token: Comprueba que el sistema maneja adecuadamente tokens inválidos.
test_reset_password_expired_token: Verifica que los tokens expirados sean rechazados.
test_reset_password_user_not_found: Asegura que el sistema responde adecuadamente cuando se intenta restablecer la contraseña para un usuario que no existe.
6. Conclusiones y Recomendaciones
6.1. Implementación de la Funcionalidad de Restablecimiento de Contraseña
Beneficios:

Mejora la Usabilidad: Permite a los usuarios recuperar el acceso a sus cuentas de manera sencilla.
Aumenta la Seguridad: Añade mecanismos para verificar la identidad del usuario antes de permitir cambios sensibles.
Reducción de Abusos: Al requerir la confirmación de email, se minimizan los intentos de restablecimiento fraudulentos.
Recomendaciones:

Implementar Reenvío de Emails de Restablecimiento: Permitir a los usuarios solicitar nuevamente el enlace de restablecimiento si no lo recibieron.
Mejorar la Gestión de Tokens: Almacenar tokens en la base de datos para invalidarlos después de su uso o expirar más rápido en caso de múltiples intentos.
Integrar Captcha: Añadir medidas como CAPTCHA en los formularios de restablecimiento para prevenir abusos automatizados.
6.2. Mejorar la Seguridad y Gestión de Usuarios
Restricción de Acceso Basada en Confirmación de Email:

Asegura que solo los usuarios que han confirmado su dirección de email puedan iniciar sesión y utilizar la API.
Implementación de Roles y Permisos:

Añadir diferentes roles (e.g., admin, user) para controlar el acceso a funcionalidades específicas dentro de la API.
6.3. Optimizar la Experiencia del Usuario en el Frontend
Flujo de Restablecimiento de Contraseña:

Proporcionar una navegación clara y mensajes informativos durante el proceso de restablecimiento.
Redirigir al usuario a páginas de éxito o login después de completar el restablecimiento.
Validaciones en el Frontend:

Asegurar que los formularios tengan validaciones adecuadas para mejorar la experiencia del usuario y reducir errores.
6.4. Ampliar las Pruebas Automatizadas
Cobertura de Casos de Uso:
Continuar añadiendo pruebas para cubrir todos los posibles escenarios y asegurar la robustez de la API.
Pruebas de Seguridad:
Implementar pruebas que verifiquen la protección contra ataques comunes como inyección SQL, XSS, y CSRF.
7. Recursos Adicionales
Flask-Mail Documentation: Flask-Mail Docs
itsdangerous Documentation: itsdangerous Docs
Flask-JWT-Extended Documentation: Flask-JWT-Extended Docs
Flask-SQLAlchemy Documentation: Flask-SQLAlchemy Docs
Flask-Migrate Documentation: Flask-Migrate Docs
Flask-Bcrypt Documentation: Flask-Bcrypt Docs
PyTest Documentation: PyTest Docs
SQLAlchemy Documentation: SQLAlchemy Docs
Flask Testing Documentation: Flask Testing Docs
GitHub Actions Documentation: GitHub Actions Docs
Heroku Documentation: Heroku Docs
OWASP Password Storage Cheat Sheet: OWASP Password Storage
Conclusión
En el Día 26, has ampliado la funcionalidad de gestión de usuarios de tu API al implementar un sistema de restablecimiento de contraseña. Esta característica mejora la usabilidad al permitir a los usuarios recuperar el acceso a sus cuentas y refuerza la seguridad al asegurar que solo los usuarios legítimos puedan cambiar sus contraseñas.

Pasos Clave Realizados:

Configuración de Funcionalidades de Restablecimiento de Contraseña:
Implementaste funciones para generar y verificar tokens seguros para restablecimiento.
Implementación de Endpoints:
Creaste /reset_password_request para solicitar el restablecimiento y /reset_password/<token> para establecer una nueva contraseña.
Actualización del Frontend:
Añadiste formularios y mensajes en el frontend para manejar el flujo de restablecimiento de contraseña.
Actualización de Pruebas Automatizadas:
Añadiste pruebas para verificar que el proceso de restablecimiento funciona correctamente y maneja adecuadamente los errores.
Recomendaciones para Continuar:

Implementar Roles y Permisos:
Añadir diferentes niveles de acceso para controlar funcionalidades específicas.
Mejorar la Seguridad de JWT:
Considerar la implementación de refresh tokens y gestionar adecuadamente la expiración de tokens.
Optimizar la Experiencia del Usuario:
Proporcionar una interfaz más intuitiva y mensajes claros durante todo el proceso de registro, login y restablecimiento de contraseña.
Ampliar las Funcionalidades de Seguridad:
Implementar medidas adicionales como CAPTCHA, limitación de intentos de login y monitoreo de actividad sospechosa.